<a href="https://colab.research.google.com/github/Praagnya/IPL-2020-Anaysis/blob/main/IPL_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline 
from plotly import tools 
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objects as go
color = sns.color_palette()



In [2]:
url = 'https://raw.githubusercontent.com/Praagnya/IPL-2020-Anaysis/main/Matches%20IPL%202020.csv'
matches = pd.read_csv(url)
matches.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1216492,2020,Abu Dhabi,09/19/2020,MI,CSK,CSK,field,normal,0,CSK,0,5,AT Rayudu,"Sheikh Zayed Stadium, Abu Dhabi",Chris Gaffaney,Virender Sharma,Sundaram Ravi
1,1216493,2020,Dubai (DSC),09/20/2020,DC,KXIP,KXIP,field,tie,0,DC,0,0,MP Stoinis,Dubai International Cricket Stadium,Anil Chaudhary,Nitin Menon,Paul Reiffel
2,1216534,2020,Dubai (DSC),09/21/2020,SRH,RCB,SRH,field,normal,0,RCB,10,0,YS Chahal,Dubai International Cricket Stadium,Anil Dandekar,Nitin Menon,Anil Chaudhary
3,1216496,2020,Sharjah,09/22/2020,RR,CSK,CSK,field,normal,0,RR,16,0,SV Samson,Sharjah Cricket Stadium,Chettithody Shamshuddin,Vineet Kulkarni,KN Ananthapadmanabhan
4,1216508,2020,Abu Dhabi,09/23/2020,KKR,MI,KKR,field,normal,0,MI,49,0,RG Sharma,"Sheikh Zayed Stadium, Abu Dhabi",Chris Gaffaney,Sundaram Ravi,Virender Sharma


In [3]:
url2 = 'https://raw.githubusercontent.com/Praagnya/IPL-2020-Anaysis/main/Deliveries%20IPL%202020.csv'
deliveries = pd.read_csv(url2)
deliveries.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1216492,1,MI,CSK,1,1,RG Sharma,Q de Kock,DL Chahar,0,0,0,0,0,0,4,0,4,NaN,NaN,NaN
1,1216492,1,MI,CSK,1,2,RG Sharma,Q de Kock,DL Chahar,0,0,0,0,0,0,1,0,1,NaN,NaN,NaN
2,1216492,1,MI,CSK,1,3,Q de Kock,RG Sharma,DL Chahar,0,0,0,0,0,0,2,0,2,NaN,NaN,NaN
3,1216492,1,MI,CSK,1,4,Q de Kock,RG Sharma,DL Chahar,0,0,0,0,0,0,4,0,4,NaN,NaN,NaN
4,1216492,1,MI,CSK,1,5,Q de Kock,RG Sharma,DL Chahar,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN


In [4]:
matches.isnull().sum()

id                 0
season             0
city               0
date               0
team1              0
team2              0
toss_winner        0
toss_decision      0
result             0
dl_applied         0
winner             0
win_by_runs        0
win_by_wickets     0
player_of_match    0
venue              0
umpire1            0
umpire2            0
umpire3            0
dtype: int64

In [5]:
batsman_grp = deliveries.groupby(["match_id", "inning", "batting_team", "batsman"])
batsmen = batsman_grp["batsman_runs"].sum().reset_index()

# Ignore the wide balls.
balls_faced = deliveries[deliveries["wide_runs"] == 0]
balls_faced = balls_faced.groupby(["match_id", "inning", "batsman"])["batsman_runs"].count().reset_index()
balls_faced.columns = ["match_id", "inning", "batsman", "balls_faced"]
batsmen = batsmen.merge(balls_faced, left_on=["match_id", "inning", "batsman"], 
                        right_on=["match_id", "inning", "batsman"], how="left")

fours = deliveries[ deliveries["batsman_runs"] == 4]
sixes = deliveries[ deliveries["batsman_runs"] == 6]


fours_per_batsman = fours.groupby(["match_id", "inning", "batsman"])["batsman_runs"].count().reset_index()
sixes_per_batsman = sixes.groupby(["match_id", "inning", "batsman"])["batsman_runs"].count().reset_index()

fours_per_batsman.columns = ["match_id", "inning", "batsman", "4s"]
sixes_per_batsman.columns = ["match_id", "inning", "batsman", "6s"]

batsmen = batsmen.merge(fours_per_batsman, left_on=["match_id", "inning", "batsman"], 
                        right_on=["match_id", "inning", "batsman"], how="left")
batsmen = batsmen.merge(sixes_per_batsman, left_on=["match_id", "inning", "batsman"], 
                        right_on=["match_id", "inning", "batsman"], how="left")
batsmen['SR'] = np.round(batsmen['batsman_runs'] / batsmen['balls_faced'] * 100, 2)

for col in ["batsman_runs", "4s", "6s", "balls_faced", "SR"]:
    batsmen[col] = batsmen[col].fillna(0)

dismissals = deliveries[ pd.notnull(deliveries["player_dismissed"])]
dismissals = dismissals[["match_id", "inning", "player_dismissed", "dismissal_kind", "fielder"]]
dismissals.rename(columns={"player_dismissed": "batsman"}, inplace=True)
batsmen = batsmen.merge(dismissals, left_on=["match_id", "inning", "batsman"], 
                        right_on=["match_id", "inning", "batsman"], how="left")

batsmen = matches[['id','season']].merge(batsmen, left_on = 'id', right_on = 'match_id', how = 'left').drop('id', axis = 1)
batsmen.head()

,season,match_id,inning,batting_team,batsman,batsman_runs,balls_faced,4s,6s,SR,dismissal_kind,fielder
0,2020,1216492,1,MI,AS Yadav,17,15,2.0,0.0,113.33,NaN,NaN
1,2020,1216492,1,MI,HH Pandya,14,9,0.0,2.0,155.56,NaN,NaN
2,2020,1216492,1,MI,JJ Bumrah,5,3,0.0,0.0,166.67,NaN,NaN
3,2020,1216492,1,MI,JL Pattinson,11,7,2.0,0.0,157.14,NaN,NaN
4,2020,1216492,1,MI,KA Pollard,18,13,1.0,1.0,138.46,NaN,NaN


# Most runs in the season 

In [6]:
import plotly.express as px
most_runs = batsmen.groupby(['batsman'])['batsman_runs'].sum()
most_runs.sort_values(ascending=False, inplace=True)
most_runs = most_runs.reset_index()
most_runs_10 = most_runs[:10]

fig = px.bar(most_runs_10, x="batsman_runs", y="batsman", orientation='h')
fig.update_layout(barmode='stack', yaxis={'categoryorder':'total ascending'})
fig.show(renderer="colab")

# Strike Rate vs Boundaries 

In [7]:
batsmen['boundary'] = batsmen['4s'] + batsmen['6s']
batsmen.head()


,season,match_id,inning,batting_team,batsman,batsman_runs,balls_faced,4s,6s,SR,dismissal_kind,fielder,boundary
0,2020,1216492,1,MI,AS Yadav,17,15,2.0,0.0,113.33,NaN,NaN,2.0
1,2020,1216492,1,MI,HH Pandya,14,9,0.0,2.0,155.56,NaN,NaN,2.0
2,2020,1216492,1,MI,JJ Bumrah,5,3,0.0,0.0,166.67,NaN,NaN,0.0
3,2020,1216492,1,MI,JL Pattinson,11,7,2.0,0.0,157.14,NaN,NaN,2.0
4,2020,1216492,1,MI,KA Pollard,18,13,1.0,1.0,138.46,NaN,NaN,2.0


In [8]:
most_boundary = batsmen.groupby('batsman')['boundary'].sum()
most_boundary.sort_values(ascending=False)
most_boundary = most_boundary.reset_index()
matches_final = pd.merge(batsmen,most_boundary,how='outer',on='batsman')
matches_final.head(2)

,season,match_id,inning,batting_team,batsman,batsman_runs,balls_faced,4s,6s,SR,dismissal_kind,fielder,boundary_x,boundary_y
0,2020,1216492,1,MI,AS Yadav,17,15,2.0,0.0,113.33,NaN,NaN,2.0,56.0
1,2020,1216508,1,MI,AS Yadav,47,28,6.0,1.0,167.86,NaN,NaN,7.0,56.0


In [9]:
avg_sr = batsmen.groupby('batsman')['SR'].mean()
avg_sr.sort_values(ascending=False)
avg_sr = avg_sr.reset_index()
avg_sr.rename(columns = {'SR':'Avg_SR'}, inplace = True)
matches_final1 = pd.merge(matches_final, avg_sr, how='outer',on='batsman')
matches_final1.head(2)

,season,match_id,inning,batting_team,batsman,batsman_runs,balls_faced,4s,6s,SR,dismissal_kind,fielder,boundary_x,boundary_y,Avg_SR
0,2020,1216492,1,MI,AS Yadav,17,15,2.0,0.0,113.33,NaN,NaN,2.0,56.0,124.15
1,2020,1216508,1,MI,AS Yadav,47,28,6.0,1.0,167.86,NaN,NaN,7.0,56.0,124.15


In [10]:
avg_sr = batsmen.groupby('batsman')['SR'].mean()
avg_sr.sort_values(ascending=False)
avg_sr = avg_sr.reset_index()
avg_sr.rename(columns = {'SR':'Avg_SR'}, inplace = True)
matches_final1 = pd.merge(avg_sr,matches_final[['boundary_y','batsman']], how='outer',on='batsman')
matches_final1.drop_duplicates(inplace=True)
matches_final2 = matches_final1[((matches_final1['Avg_SR']>110) & (matches_final1['boundary_y'] > 20))]
matches_final2.head(2)

,batsman,Avg_SR,boundary_y
0,AB de Villiers,143.5425,57.0
51,AS Yadav,124.1500,56.0


In [11]:
fig = px.scatter(matches_final2, x=matches_final2['Avg_SR'], y=matches_final2['boundary_y'], text=matches_final2['batsman'], log_x=True, size_max=10)

fig.update_traces(textposition='top center')

fig.update_layout(
    height=800,
    title_text='Strike Rate vs No of boundaries'
)

fig.show(renderer = 'colab')

# Impact of toss on result of the match

In [12]:
toss = matches.groupby(['toss_winner']).winner.value_counts().reset_index(name = 'count')
toss['result'] = np.where(toss.toss_winner == toss.winner, 'won', 'lost')
toss_result = toss.groupby(['toss_winner','result'])['count'].sum().reset_index()

In [13]:
toss_result

,toss_winner,result,count
0,CSK,lost,4
1,CSK,won,4
2,DC,lost,7
3,DC,won,3
4,KKR,lost,4
5,KKR,won,2
6,KXIP,lost,3
7,KXIP,won,1
8,MI,lost,3
9,MI,won,5


In [14]:
fig = px.bar(toss_result, x="toss_winner", y="count",color="result",barmode='group', height=500)
fig.update_layout(title_text='Match Result after Winning the Toss')
fig.show(renderer = 'colab')

It looks like Mumbai Indians and Sunrisers Hyderabad have made the most out of toss. 

# Strike Rate vs 6s

In [15]:
fig = px.scatter(batsmen.query("balls_faced>15"), x='SR', y="6s",size = '4s',
	          color="batsman",
                 hover_name='batsman', log_x=True, size_max=60)
fig.show(renderer = 'colab')



# Work in progress........